In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import keras.preprocessing.image as kim
from shutil import copyfile
from scipy.misc import imsave


In [ ]:
start_directory = ''
t_path = 'train'
v_path = 'train'
augmented_t_path = 'pretrained_test/augmented_train_final1/'
augmented_v_path = 'pretrained_valid/augmented_valid_final/'
image_height = 128
image_width = 128


In [ ]:
df = pd.read_csv('train.csv')


In [ ]:
cls_fr = df.groupby("Id",as_index = False)["Image"].count()
cls_fr = cls_fr.rename(columns = {"Image":"count_image"})
cls_fr=cls_fr.sort_values('count_image')
cls_fr=cls_fr[['Id','count_image']].query('Id!="new_whale"')
df2=cls_fr.tail(20)
df2

In [ ]:

lst=[]
lst=df2['Id']
for x in lst:
    print(x)

In [ ]:


class_dictionary = convert_dict('train.csv') #class_dictionary == label map
classes = set(class_dictionary.values())

In [ ]:

def convert_dict(l_file):
    dictionary = {}  
    with open(l_file, 'r') as lab_file:
        _ = lab_file.readline()  
        for line in lab_file:
            whale_name, whale_class = line.strip().split(',')
            dictionary[whale_] = whale_class
    return dictionary

In [ ]:
for cls in classes:
    sub_folder = os.path.join('training_dataset/', cls)
    if not os.path.exists(sub_folder):
        os.mkdir(sub_folder)

In [ ]:
def moving_related_classes(path1,path2):
    for wanted_folders in lst:
        directry=os.path.join(path1,wanted_folders)
        shutil.move(directry, path2)
    

In [ ]:

p1='training_dataset/'
p2='training_dataset_30'
moving_related_classes(p1,p2)
p3='validation'
p4='validation_30'
moving_related_classes(p3,p4)


In [ ]:
train='train/train'
whale_images = os.listdir(train)  #whale images=image names
for img_1 in whale_images:
    image_file = os.path.join(train, img_1)
    if os.path.isfile(image_file):
        img = resize_img(image_file, dim=(image_height, image_width))
    
        label = class_dictionary[img_1]
        if len(os.listdir(os.path.join('training_dataset/', label))) <14:
            cur_lab_cnt = len(os.listdir(os.path.join('training_dataset/', label)))
            save_name = str(cur_lab_cnt + 1) + '.jpg'
            imsave(os.path.join(base_data_path,'training_dataset/', label, save_name), img)
        else:
            cur_cnt = len(os.listdir(os.path.join('validation/', label)))
            save_name = str(cur_cnt + 1) + '.jpg'
            imsave(os.path.join('validation/', label, save_name), img) 


In [ ]:
def resize_img(file_path, dim=None):
    x = np.array(imread(file_path), dtype=np.float32)
    if len(x.shape) < 3 or x.shape[-1] < 3:
        x = np.squeeze(x)
        x = np.stack((x,) * 3, axis=-1)
    elif x.shape[-1] > 3:
        x = x[:, :, :3]  
    if dim is not None:
        x = imresize(x, dim)
    x = x / 255
    return x

In [ ]:

def aug_rotate(img_array, n=6, angle=10):
    aug1 = [ker.random_rotation(img_array, angle,row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')for _ in range(n)]   
    return aug1

In [ ]:
def aug_shear(img_array, n=6):
    aug2 = [ker.random_shear(img_array, intensity=7, 
                        row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
                        for _ in range(n)]
    return aug2


In [ ]:

def aug_zoom(img_array, n=6,wt=1.5, ht=0.8):
    aug3 = [kim.random_zoom(img_array, zoom_range=(wt, ht), row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
                        for _ in range(n)]    
    return aug3

In [ ]:
def aug_shift(img_array, n=6):
    aug4 = [ker.random_shift(img_array, wrg=0.1, hrg=0.2, 
                        row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
                        for _ in range(n)]
    
    return aug4

In [ ]:

import os
import numpy as np
import matplotlib.pyplot as plt
import keras.preprocessing.image as kim
from shutil import copyfile
from scipy.misc import imsave
import pandas as pd
from keras import optimizers
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

import cv2

In [ ]:


def preprocess_images(path, num_iterations=4):
    no_img=60
    preprocessing = [aug_rotate,aug_shift, aug_shear, aug_zoom]
    labels = os.listdir(path)
    counter_label = 1
    for label in labels:
        class_path = os.path.join(path, label)
        for i in preprocessing:
            images = os.listdir(class_path)
            cnt = len(images) + 1            
            for image in images:
                img_name = os.path.join(class_path, image)
                image_array = cv2.imread(img_name)
                pre_img = i(image_array, num_iterations)               
                for kth_img in pre_img:
                    if cnt > no_img:
                        break
                    imsave(os.path.join(class_path, str(cnt)+'.jpg'), kth_img)
                    cnt += 1
        counter_label += 1


In [ ]:
preprocess_images(augmented_t_path)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
size_of_batch=32
im_col=128
im_row=128
im_shape=(im_row,im_col,3)

epochs=iterations
augmented_t_path = 'cnn_basic_noprepro/augmented_train_final1/'
augmented_v_path = 'cnn_basic_noprepro/augmented_valid_final/'
#check here tomorrow
T_Data = ImageDataGenerator(
        rescale=1./255)
Test_gen = ImageDataGenerator(rescale=1./255)
t_images = T_Data.flow_from_directory(
        'cnn_basi c_noprepro/augmented_train_final1/',target_size=(128, 128),
        batch_size=size_of_batch,
        class_mode='categorical')

v_images = Test_gen.flow_from_directory(
        'cnn_basic_noprepro/augmented_valid_final/',target_size=(128, 128),
        batch_size=size_of_batch,
        class_mode='categorical')


In [ ]:
t_images[0][0].shape

In [ ]:
size_of_batch = 32
number_of_samples = 100
v_steps = 5
filter1 = 32
filter2 = 64
conv_layer_1_size = 3
conv_layer_2_size = 2
pooling = 3
classes_num = 30
rateOfLearning = 0.001
img_height = 128
img_width = 128
iterations= 20;
v_images[0][0].shape

In [ ]:
im_shape


In [ ]:
BasicCNNModel = Sequential()
BasicCNNModel.add(Convolution2D(filter1, conv_layer_1_size, conv_layer_1_size, border_mode ="same", input_shape=(im_col, im_row, 3),name="Convolution_layer_1"))
BasicCNNModel.add(Activation("relu"))
BasicCNNModel.add(MaxPooling2D(pool_size=(pooling, pooling),name="Maxpooling_layer_1"))

BasicCNNModel.add(Convolution2D(filter2, conv_layer_2_size, conv_layer_2_size, border_mode ="same",name="Convolution_layer_2"))
BasicCNNModel.add(Activation("relu"))
BasicCNNModel.add(MaxPooling2D(pool_size=(pooling, pooling), dim_ordering='th',name="Maxpooling_layer_2"))

BasicCNNModel.add(Flatten())
BasicCNNModel.add(Dense(256,name="Dense_layer_1"))
BasicCNNModel.add(Activation("relu"))
BasicCNNModel.add(Dropout(0.5))
BasicCNNModel.add(Dense(classes_num, activation='softmax',name="Dense_layer_2"))

BasicCNNModel.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=rateOfLearning),
              metrics=['accuracy'])

In [1]:
T_Data = ImageDataGenerator(
    rescale=1. / 255)


NameError: name 'ImageDataGenerator' is not defined

In [ ]:
Test_data = ImageDataGenerator(rescale=1. / 255)
augmented_t_path = 'cnn_basic_noprepro/augmented_train_final1/'
augmented_v_path = 'cnn_basic_noprepro/augmented_valid_final/'
t_images = T_Data.flow_from_directory(
    augmented_t_path,
    target_size=(img_height, img_width),
    batch_size=size_of_batch,
    class_mode='categorical')

v_images = Test_data.flow_from_directory(
    augmented_v_path,
    target_size=(img_height, img_width),
    batch_size=size_of_batch,
    class_mode='categorical')

In [ ]:
BasicCNNModel.summary()

In [ ]:
tensorboard=TensorBoard(
    log_dir=r'logs\{}'.format('2ndLayer'),
    write_graph=True,
    write_grads=True,
    write_images=True
)
PlotInfo=BasicCNNModel.fit_generator(
    t_images,
    samples_per_epoch=number_of_samples,
    epochs=iterations,
    validation_data=v_images,
    validation_steps=v_steps)

In [ ]:
target_dir = './models/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
BasicCNNModel.save('./models/model.h5')
BasicCNNModel.save_weights('./models/weights.h5')

In [ ]:

c=train_generator.class_indices
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
file=r'C:\Users\pooja\AnacondaProjects\ML project\test\w_f19faeb_1.jpg'
x = load_img(file, target_size=(img_width,img_height))
x = img_to_array(x)
x = np.expand_dims(x, axis=0)
array = BasicCNNModel.predict(x)
ind = np.unravel_index(np.argmax(array, axis=None), array.shape)
print(list(c.keys())[list(c.values()).index(ind[1])])

In [ ]:
print(BasicCNNModel.summary())

In [ ]:
plt.plot(PlotInfo.history['acc'])
plt.plot(PlotInfo.history['val_acc'])
plt.title('ACCURACY')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.plot(PlotInfo.history['loss'])
plt.plot(PlotInfo.history['val_loss'])
plt.title('LOSS')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()